# Atlantic Estuarine Fish Habitat

This indicator was created for the 2022 Base Blueprint. It was modified from the 2021 SA indicator by change the extent to cover a larger area. However, it was later decided to not use it in the full extent in zonation. I'm saving this copy so we can revisit the extent in 2023

Created by Amy Keister, last run by Amy Keister 1 August, 2022. It tookk 12 minutes to run.

In [20]:
import os
import arcpy

In [21]:
import time
start = time.time()

In [22]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\AtlanticEst\AtlanticEstua.gdb"

In [23]:
# define final indicator outputs
# this version will be for Brendan and for zonaiton because it has the full raster extent of the entire SE Blueprint
#OutMaxE = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\AtlanticEst\AtlanticEstuarineFishHabitatMaxE.tif"
# this version will be for the download package 
OutMinE = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\AtlanticEst\AtlanticEstuarineFishHabitat.tif"

In [32]:
# define rasters used for cell size, extent, and snapping
Rextent= r"D:\SE_Blueprint_2023\2_FullExtent\SEBlueprint2023ExtentVIPR6.tif"

In [25]:
# define inputs
sa= r"F:\GIS_DATA\DecisionSupportTools\ACFHP_FishHabitatConservationAreaMapping\South Atlantic Estuarine Analysis\data\v101\estuarine.gdb\Estuarine_Units_1sqkm_Northern05082018"
ma= r"F:\GIS_DATA\DecisionSupportTools\ACFHP_FishHabitatConservationAreaMapping\Mid-Atlantic Estuarine Analysis\data\v101\finallayers.gdb\hexagons_Mid_Atlantic"
fl= r"F:\GIS_DATA\DecisionSupportTools\ACFHP_FishHabitatConservationAreaMapping\South Florida Estuarine Analysis\data\v101\estuarine.gdb\Estuarine_Units_1sqkm_Southern05082018"
#MarineRaster= r"D:\SE_Blueprint_2023\2_MarineExtents\Marine_Extent_v1.tif"
#base= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"
#SEextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\SE_Blueprint_2022.tif"

## Start Analysis

In [26]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [28]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\AtlanticEst\AtlanticEstua.gdb


## Start Analysis

In [27]:
# make a copy of the south atlantic layer for edits
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.management.CopyFeatures(sa, "sa", '', None, None, None)

In [9]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent):
    arcpy.conversion.FeatureToRaster(in_features="sa", field="FINALSCORE", out_raster="saR", cell_size=Rextent)

In [10]:
# make a copy of the mid atlantic layer for edits
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.management.CopyFeatures(ma, "ma", '', None, None, None)

In [11]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent):
    arcpy.conversion.FeatureToRaster(in_features="ma", field="TotalPoints", out_raster="maR", cell_size=Rextent)

In [12]:
# make a copy of the florida layer for edits
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.management.CopyFeatures(fl, "fl", '', None, None, None)

In [13]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent):
    arcpy.conversion.FeatureToRaster(in_features="fl", field="FINALSCORE", out_raster="flR", cell_size=Rextent)

In [18]:
# i'm getting an error with my cell statistics, it says invalid output extent
# I'm testing to see if using this helps
arcpy.analysis.Union([sa,ma,fl], "samaflR", "ONLY_FID", None, "GAPS")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\AtlanticEst\\AtlanticEstua.gdb\\samaflR'>

In [20]:
# mosaic to new raster is creating an output with bands of missing data switching to a new method to combine
# use cell statistics to combine the raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="samaflR", snapRaster=Rextent):
    out_raster = arcpy.sa.CellStatistics("saR;maR;flR1", "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("saRmaRflR")

In [22]:
# reclassify
out_raster = arcpy.sa.Reclassify("saRmaRflR", "Value", "0 0;10 1;20 2;30 3;40 4;50 5;60 6;70 7;80 8", "DATA"); out_raster.save("indicator")

### Make a mask layer

Since we didn't make a layer depicting the full extent of the continenial Southast blueprint, I have to make that now

In [23]:
# use cell statistics to combine the Marine raster and base blueprint extent
# This will be used to remvoe deep marine areas outside the blueprint
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
#    out_raster = arcpy.sa.CellStatistics([MarineRaster, base], "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("FullExtent")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [24]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 8:
        return '8 = Final score of 80 (areas of excellent fish habitat)'
    elif value == 7:
        return '7 = Final score of 70 (areas of excellent fish habitat)' 
    elif value == 6:
        return '6 = Final score of 60 (restoration opportunity areas)' 
    elif value == 5:
        return '5 = Final score of 50 (restoration opportunity areas)'
    elif value == 4:
        return '4 = Final score of 40 (restoration opportunity areas)' 
    elif value == 3:
        return '3 = Final score of 30 (restoration opportunity areas)' 
    elif value == 2:
        return '2 = Final score of 20 (restoration opportunity areas)'
    elif value == 1:
        return '1 = Final score of 10 (degraded areas of opportunity)' 
    elif value == 0:
        return '0 = Final score of 0 (degraded areas of opportunity)' 
"""

In [25]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("indicator", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\AtlanticEst\\AtlanticEstua.gdb\\indicator'>

In [29]:
# set code block for next step
codeblock = """
def Reclass1(Value):
    if Value == 8:
        return 8
    elif Value == 7:
        return 8
    elif Value == 6:
        return 32
    elif Value == 5:
        return 61
    elif Value == 4:
        return 95
    elif Value == 3:
        return 132
    elif Value == 2:
        return 170
    elif Value == 1:
        return 200
    elif Value == 0:
        return 219
    else:
        return 0

def Reclass2(Value):
    if Value == 8:
        return 64
    elif Value == 7:
        return 98
    elif Value == 6:
        return 129
    elif Value == 5:
        return 161
    elif Value == 4:
        return 188
    elif Value == 3:
        return 208
    elif Value == 2:
        return 222
    elif Value == 1:
        return 234
    elif Value == 0:
        return 241
    else:
        return 0

def Reclass3(Value):
    if Value == 8:
        return 129
    elif Value == 7:
        return 165
    elif Value == 6:
        return 184
    elif Value == 5:
        return 201
    elif Value == 4:
        return 205
    elif Value == 3:
        return 194
    elif Value == 2:
        return 182
    elif Value == 1:
        return 195
    elif Value == 0:
        return 213
    else:
        return 0
"""

In [30]:
# calculate Red field
arcpy.management.CalculateField("indicator", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("indicator", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("indicator", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'indicator'>

In [33]:
# make the version for the download package that is the smallest extent nodata bounding box that is possible with these data
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("MinE")

In [34]:
# export as .tif
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("MinE", OutMinE, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [30]:
# make a version for Brendan with full SE blueprint extent bounding box
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
#    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("MaxE")

In [31]:
# export as .tif
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
#    arcpy.management.CopyRaster("MaxE", OutMaxE, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [32]:
end = time.time()
print(end - start)

3004.669203519821
